In [12]:
from ultralytics import YOLO

model = YOLO('yolov8m.yaml')


                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384

In [5]:
import os
from tqdm import tqdm


root = './dataset/train/'
img_fns = [fn for fn in os.listdir(root) if fn.endswith('.png')]

for fn in tqdm(img_fns[:int(len(img_fns) * 0.8)]):
    os.remove(os.path.join(root, fn))
    os.remove(os.path.join(root, fn.replace('.png', '.txt')))

100%|████████████████████████████████████| 1222/1222 [00:00<00:00, 52736.28it/s]


In [6]:
from torchinfo import summary

feature_extractor = model.model.model[:10]

summary(feature_extractor, input_size=(1, 3, 512, 512), device='cpu', depth=0, col_names=['input_size', 'output_size', 'num_params'])

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #
Sequential                                    [1, 3, 512, 512]          [1, 576, 16, 16]          11,855,856
Total params: 22,684,272
Trainable params: 22,684,272
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 12.32
Input size (MB): 3.15
Forward/backward pass size (MB): 323.62
Params size (MB): 47.42
Estimated Total Size (MB): 374.19

In [7]:
import cv2
import os
from tqdm import tqdm

root = './dataset/train/'

imgs = [cv2.imread(os.path.join(root, fn)) for fn in tqdm(os.listdir(root)) if fn.endswith('.png')]

len(imgs)

100%|████████████████████████████████████████| 613/613 [00:01<00:00, 593.53it/s]


306

In [8]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from torch import nn
import torch
from torchinfo import summary


class SSLPatchPretrainingDataset(Dataset):
    def __init__(self, imgs):
        super(SSLPatchPretrainingDataset, self).__init__()

        self.imgs = imgs
        self.__construct_patches()
        # мб потом что-нибудь добавлю для аугментации
        self.transforms = T.Compose([T.ToTensor(),
                                     T.Resize((224, 224), T.InterpolationMode.BICUBIC, antialias=True)])

    def __construct_patches(self):
        self.inner_patches = list()
        self.outer_patches = list()

        for img in self.imgs:
            y1 = int(img.shape[0] * 1/3)
            y2 = int(img.shape[0] * 2/3)
            y3 = img.shape[0]

            x1 = int(img.shape[1] * 1/3)
            x2 = int(img.shape[1] * 2/3)
            x3 = img.shape[1]

            self.outer_patches.append([img[:y1, :x1], img[y1:y2, :x1], img[y2:y3, :x1],
                                       img[:y1, x1:x2], img[y2:y3, x1:x2],
                                       img[:y1, x2:x3], img[y1:y2, x2:x3], img[y2:y3, x2:x3],])
            self.inner_patches.append(img[y1:y2, x1:x2])


    def __len__(self):
        return len(self.imgs) * 8

    def __getitem__(self, idx):
        img_idx = idx % len(self.imgs)
        outer_patch_idx = idx // len(self.imgs)

        inner_patch = self.inner_patches[img_idx]
        outer_patch = self.outer_patches[img_idx][outer_patch_idx]

        inner_patch = self.transforms(inner_patch)
        outer_patch = self.transforms(outer_patch)

        return inner_patch, outer_patch, outer_patch_idx


class SSLPatchPretrainingModel(nn.Module):
    def __init__(self, backbone):
        super(SSLPatchPretrainingModel, self).__init__()

        self.backbone = backbone
        self.head = nn.Sequential(nn.AvgPool2d(7), # global average pooling
                                  nn.Flatten(start_dim=1), # squeezing extra dims
                                  nn.Linear(576*2, 8),
                                  nn.Softmax(dim=-1))

    def forward(self, xi, xo):
        fi = self.backbone(xi)
        fo = self.backbone(xo)

        # x2 (B, C, W, H)
        f = torch.cat([fi, fo], dim=1)

        y = self.head(f)

        return y


feature_extractor = model.model.model[:10]
pretraining_model = SSLPatchPretrainingModel(feature_extractor)

summary(pretraining_model, input_size=[(128, 3, 224, 224), (128, 3, 224, 224)], device='cpu', depth=1)

Layer (type:depth-idx)                             Output Shape              Param #
SSLPatchPretrainingModel                           [128, 8]                  --
├─Sequential: 1-1                                  [128, 576, 7, 7]          11,855,856
├─Sequential: 1-2                                  [128, 576, 7, 7]          (recursive)
├─Sequential: 1-3                                  [128, 8]                  9,224
Total params: 22,693,496
Trainable params: 22,693,496
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 603.70
Input size (MB): 154.14
Forward/backward pass size (MB): 15857.23
Params size (MB): 47.46
Estimated Total Size (MB): 16058.83

In [9]:
import torch
from tqdm import trange

from torch.optim import NAdam
from torch.nn import CrossEntropyLoss
from torchmetrics import Accuracy


epochs = 10
batch_size = 128
early_stopping_patience = 4


device = 'cuda' if torch.cuda.is_available() else 'cpu'

pretraining_model.train()
pretraining_model = pretraining_model.to(device)

optimizer = NAdam(pretraining_model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss()
accuracy = Accuracy('multiclass', num_classes=8).to(device)

train_dataset = SSLPatchPretrainingDataset(imgs)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=2, pin_memory=True, shuffle=True)

train_loss_history = list()

for epoch in trange(epochs):
    train_loss = 0
    train_accuracy = 0
    for (xi, xo, y) in train_dataloader:
        optimizer.zero_grad()

        xi = xi.to(device)
        xo = xo.to(device)
        y = y.to(device)

        y_pred = pretraining_model(xi, xo)

        loss = criterion(y_pred, y)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_accuracy += accuracy(y_pred, y).item()
    print(f'Epoch {epoch}/{epochs}. Train loss: {round(train_loss / len(train_dataloader), 6)}, Train accuracy: {round(train_accuracy / len(train_dataloader), 6)}')

    train_loss_history.append(train_loss)
    if min(train_loss_history) < min(train_loss_history[-early_stopping_patience:]):
        break

 10%|████▍                                       | 1/10 [00:07<01:04,  7.19s/it]

Epoch 0/10. Train loss: 2.082709, Train accuracy: 0.124219


 20%|████████▊                                   | 2/10 [00:14<00:56,  7.06s/it]

Epoch 1/10. Train loss: 2.084385, Train accuracy: 0.113281


 30%|█████████████▏                              | 3/10 [00:21<00:48,  6.97s/it]

...

Epoch 7/10. Train loss: 2.080603, Train accuracy: 0.154297


 90%|███████████████████████████████████████▌    | 9/10 [01:01<00:06,  6.75s/it]

Epoch 8/10. Train loss: 2.074937, Train accuracy: 0.147656


100%|███████████████████████████████████████████| 10/10 [01:08<00:00,  6.82s/it]

Epoch 9/10. Train loss: 2.086304, Train accuracy: 0.127734


In [10]:
for i in range(10):
    model.model.model[i] = pretraining_model.backbone[i]

assert model.model.model[0] == pretraining_model.backbone[0]

In [13]:
model.train(data='/home/peter/pgi_course_2023/dataset/data.yml',
            epochs=500,
            patience=25,
            batch=64,
            device='cuda',
            imgsz=512,
            workers=6,
            mosaic=0,
            scale=0.1)

Ultralytics YOLOv8.0.232 🚀 Python-3.11.5 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3090, 24238MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/home/peter/pgi_course_2023/dataset/data.yml, epochs=500, time=None, patience=25, batch=64, imgsz=512, save=True, save_period=-1, cache=False, device=cuda, workers=6, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

train: Scanning /home/peter/pgi_course_2023/dataset/train.cache... 306 images, 0
val: Scanning /home/peter/pgi_course_2023/dataset/val.cache... 765 images, 0 bac


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000227, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
500 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/500      16.5G      5.426       6.15      4.297        388        512: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all        765       6835          0          0          0          0


...

                   all        765       6835       0.59      0.292      0.323      0.216
Stopping training early as no improvement observed in last 25 epochs. Best results observed at epoch 180, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



205 epochs completed in 0.334 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.1MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.1MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.232 🚀 Python-3.11.5 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3090, 24238MiB)
YOLOv8m summary (fused): 218 layers, 25862920 parameters, 0 gradients, 78.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        765       6835      0.608      0.281      0.328      0.223
                ground        765         53      0.696      0.208      0.229       0.13
          WD CB 3-pole        765        146      0.945      0.658      0.753      0.539
              CB 1pole        765        919      0.606      0.571      0.588      0.331
              CB 3pole        765        716       0.73      0.589      0.599      0.381
direct connection counter        765        222       0.84      0.572      0.686      0.505
       MD WD CB 3-pole        765         49      0.606      0.265      0.252      0.155
fuse switch disconnector        765         46      0.852      0.391      0.404      0.319
   current transformer        765        100      0.681      0.427      0.485      0.267
reactive power compensation device        765          6          0          0     0.0016    0.00096
              CB 4pole        765         21      0.852      0.238       0.33      0.222
    

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 36, 37, 38])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f59dc53dd50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.0380